In [1]:
%load_ext autoreload

In [4]:
# check installed version (must be >3.0)
import pycaret
pycaret.__version__

'3.1.0'

In [5]:
%autoreload 
import sys
sys.path.append('../')

import numpy as np
import pandas as pd

from pycaret.regression import *

# Import data

In [10]:
test = pd.read_csv('../data_feature_enigineering/X_test_selected_preprocessed_mean.csv')
test.head()

,date_forecast,direct_rad:W,sun_elevation:d,visibility:m,pressure_50m:hPa,relative_humidity_1000hPa:p,day_of_year,air_density_2m:kgm3,cloud_base_agl:m,ceiling_height_agl:m,wind_speed_u_10m:ms,sun_azimuth:d,wind_speed_v_10m:ms,day,wind_speed_10m:ms,total_cloud_cover:p,fresh_snow_24h:cm,fresh_snow_6h:cm,fresh_snow_1h:cm,building
0,2023-05-01 00:00:00,0.00,-10.5410,29907.500,1006.800,80.275,121,1.28675,1061.5500,912.7000,2.100,16.02650,3.35,1,3.950,74.950,0.0,0.0,0.0,0
1,2023-05-01 01:00:00,0.00,-7.8945,29519.074,1006.300,79.825,121,1.28600,1075.1001,1482.1000,1.925,30.49725,3.30,1,3.825,77.475,0.0,0.0,0.0,0
2,2023-05-01 02:00:00,0.00,-3.8155,31009.125,1005.800,78.000,121,1.28375,1200.4000,1791.3000,1.750,44.51725,3.20,1,3.650,88.100,0.0,0.0,0.0,0
3,2023-05-01 03:00:00,2.10,1.4125,34552.500,1005.225,75.625,121,1.28200,1179.8500,2312.8750,1.450,58.08300,3.15,1,3.500,68.600,0.0,0.0,0.0,0
4,2023-05-01 04:00:00,25.45,7.4685,35483.875,1004.750,74.225,121,1.28100,920.0500,2198.2998,1.300,71.34100,3.05,1,3.325,66.300,0.0,0.0,0.0,0


# Load Model

In [11]:
model = load_model('../models_training/01H_pycaret_lgbm_mean_with_feature_eng')

Transformation Pipeline and Model Successfully Loaded


In [12]:
model

Pipeline(memory=Memory(location=None),
         steps=[('clean_column_names',
                 TransformerWrapper(transformer=CleanColumnNames())),
                ('trained_model', LGBMRegressor(n_jobs=-1, random_state=1))])

In [13]:
predictions = predict_model(model, data=test)

In [14]:
predictions.head()

,date_forecast,direct_rad:W,sun_elevation:d,visibility:m,pressure_50m:hPa,relative_humidity_1000hPa:p,day_of_year,air_density_2m:kgm3,cloud_base_agl:m,ceiling_height_agl:m,...,sun_azimuth:d,wind_speed_v_10m:ms,day,wind_speed_10m:ms,total_cloud_cover:p,fresh_snow_24h:cm,fresh_snow_6h:cm,fresh_snow_1h:cm,building,prediction_label
0,2023-05-01 00:00:00,0.000000,-10.5410,29907.500000,1006.799988,80.275002,121,1.28675,1061.550049,912.700012,...,16.026501,3.35,1,3.950,74.949997,0.0,0.0,0.0,0,3.909937
1,2023-05-01 01:00:00,0.000000,-7.8945,29519.074219,1006.299988,79.824997,121,1.28600,1075.100098,1482.099976,...,30.497250,3.30,1,3.825,77.474998,0.0,0.0,0.0,0,3.909937
2,2023-05-01 02:00:00,0.000000,-3.8155,31009.125000,1005.799988,78.000000,121,1.28375,1200.400024,1791.300049,...,44.517250,3.20,1,3.650,88.099998,0.0,0.0,0.0,0,4.346683
3,2023-05-01 03:00:00,2.100000,1.4125,34552.500000,1005.224976,75.625000,121,1.28200,1179.849976,2312.875000,...,58.083000,3.15,1,3.500,68.599998,0.0,0.0,0.0,0,90.433406
4,2023-05-01 04:00:00,25.450001,7.4685,35483.875000,1004.750000,74.224998,121,1.28100,920.049988,2198.299805,...,71.341003,3.05,1,3.325,66.300003,0.0,0.0,0.0,0,314.710758


In [15]:
predictions_kaggle = predictions['prediction_label']
predictions_kaggle

0         3.909937
1         3.909937
2         4.346683
3        90.433406
4       314.710758
           ...    
2155     44.014600
2156     14.262980
2157      2.819209
2158      3.215317
2159      2.665111
Name: prediction_label, Length: 2160, dtype: float64

In [17]:
# count negative predictions
neg_count = 0
for i in predictions_kaggle:
    if i < 0:
        neg_count += 1
neg_count


13

In [18]:
# set all negative predictions to 0
predictions_kaggle[predictions_kaggle < 0] = 0

In [16]:
predictions_kaggle.to_csv('../data_results/02H_pycaret_lgbm_mean_with_feature_eng.csv', index=True, header='predictions')

In [18]:
tuned_model = tune_model(model, search_library='optuna', optimize='MAE')

ValueError: _CURRENT_EXPERIMENT global variable is not set. Please run setup() first.